# Práctica 6: Preprocesamiento y extracción de características
### Alumnos:
- Angel Langdon Villamayor
- Ignacio Cano Navarro

## Librerias utilizadas

In [1]:
import os
import re

import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# necessary packages
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /Users/angel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/angel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Preprocesamiento
En el preprocesamiento de texto se ha hecho lo siguiente:

- Convertido todo el texto a minúsculas.
- Se han eliminado las stopwords.
- Se han sustituido guiones y barras bajas por espacios.
- Se han eliminado urls, usuarios y emojis.
- Los hashtags se han dejado porque hemos pensado que pueden aportar información relevante. Pero posteriormente, en el caso de que empeoren las predicciones serán eliminados.
- Se han seleccionado solo las palabras, de esta forma se eliminan signos de puntuación y cualquier otro caracter extraño.
- Por último, se ha realizado un stemming de las palabras para bajarlas todas al mismo nivel.

NOTA: Al realizar el stemming, las palabras se han tokenizado, por ello, dobles espacios, retornos de carro, etc... han sido eliminados.

Todo este preprocesamiento se ha realizado ya que llevando todo el texto al mismo nivel, reduciendo el corpus dejando solamente las palabras más importantes y también cogiendo sus "raíces" hace que el diccionario de palabras se reduzca drásticamente y esto ayuda a que los clasificadores que utilicemos posteriormente sean mejores.

In [4]:
# Preprocessing
def delete_stop_words(comment):
    spanish_stopwords = stopwords.words("spanish")
    return " ".join([w for w in comment.split() if w not in spanish_stopwords])

def steam(text, stemmer):
    stemmed_text = [stemmer.stem(word) for word in word_tokenize(text)]
    return " ".join(stemmed_text)

def clean_text_column(df, col, stemmer):
    """Normalizes a string column to have a processed format 
    Arguments:
      df (pd.DataFrame): the dataframe that contains the column to normalize
      col (str): the dataframe column to normalize
      steammer (nltk.steam.SnowballStammer): the steammer to use for 
          steamming the text
    Returns:
      The dataframe with the preprocessed column
    """
    df = df.copy() # copy the dataframe avoid modifying the original
    # Make the comments to lowercase 
    df[col] = df[col].str.lower()
    # Delete the stop words
    df[col] = [delete_stop_words(c) for c in df[col]]
    # Replace underscores and hyphens with spaces 
    df[col] = df[col].str.replace("_", " ")
    df[col] = df[col].str.replace("-", " ")
    # Create the regex to delete the urls, usernames and emojis
    urls = r'https?://[\S]+'
    users = r'@[\S]+'
    emojis = r'[\U00010000-\U0010ffff]'
    # Join the regex
    expr = f'''({"|".join([urls,
                           users,
                           emojis])})'''
    # Replace the urls, users and emojis with empty string
    df[col] = df[col].str.replace(expr, "", regex=True)                      
    # Get only the words of the text
    df[col] = df[col].str.findall("\w+").str.join(" ")
    # Delete the numbers
    df[col] = df[col].str.replace("[0-9]+", "",regex=True)
    # Steam the words of the text for each text in the specified column
    df[col] = [steam(c, stemmer) for c in  df[col]]
    return df


# Initialize the steammer to Spanish language
steammer = SnowballStemmer('spanish')
# read the data
df_original = pd.read_csv("train.csv") 
# Normalize the "comment" column
df = clean_text_column(df_original, "comment", steammer)
df.head()


,topic,thread_id,comment_id,reply_to,comment_level,comment,argumentation,constructiveness,positive_stance,negative_stance,...,target_group,stereotype,sarcasm,mockery,insult,improper_language,aggressiveness,intolerance,toxicity,toxicity_level
0,CR,0_000,0_002,0_002,1,pens zum rest,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
1,CR,0_001,0_003,0_003,1,gust afeit sec gent,0,0,0,0,...,1,1,1,1,0,0,0,0,1,1
2,CR,0_002,0_004,0_004,1,asi gust mat alta mar mas inmigr asi porfavor,0,0,0,0,...,1,0,0,0,0,0,1,1,1,2
3,CR,0_003,0_005,0_005,1,loss mas valient mejor cort cabez vosotr socia...,0,0,0,0,...,1,0,1,1,0,0,0,0,1,1
4,CR,0_004,0_006,0_006,1,costumbr,0,0,0,0,...,1,1,0,0,0,0,0,0,1,1


Comprobamos si el preprocesamiento ha funcionado tal y como se había mencionado en el primer apartado.

In [5]:
for i, (comment, comment_original) in enumerate(zip(df["comment"], df_original["comment"])):
  print(comment_original)
  print(comment)
  print()
  if i == 10:
    break



Pensó: Zumo para restar.
pens zum rest

Como les gusta el afeitado en seco a esta gente.
gust afeit sec gent

asi me gusta, que se maten entre ellos y en alta mar. Mas inmigrantes asi porfavor
asi gust mat alta mar mas inmigr asi porfavor

Loss mas valientes, los que mejor cortan nuestras cabezas, Para vosotros, socialistas, izquierdistas, y no racistas, 
loss mas valient mejor cort cabez vosotr social izquierd racist

Costumbres...
costumbr

lastima que no se volvio loco del todo y se suicido de paso..
lastim volvi loc suic pas

los mejores mas preparados para cortar calabacines, este trabaja nada mas tocar tierra.
mejor mas prepar cort calabacin trabaj mas toc tierr

Los más preparados
prepar

hombre preparado viene para currar en un matadero, o dejarle de cocinero jefe en la sede del psoe
hombr prepar vien curr matader dej cociner jef sed pso

Los detuvieron en ronda malaga, un saludo
detuv rond malag salud

piensan que deben sacrificar un alma a cambio de sobrevivir. El otro día un

## Extracción de características
Los algorítmos de machine learning aprenden de un conjunto de características de los datos de train. Pero el problema es que los algoritmos de machine learning no pueden trabajar con el texto directamente. Por ello, necesitamos utilizar técnicas de extración de características para convertir el texto en matrices o vectores de características.


## Bag of words
Es una forma fácil y efectiva de transformar texto en una representación numérica.

En este esquema se tokeniza cada texto del corpus.
Después se obtiene un vocabulario a partir de todos los tokens (palabras en minúsculas) encontrados en el corpus completo.
Cada texto se representa con un vector, donde cada entrada representa las veces que aparece una palabra del vocabulario en el texto (en su versión más siemple un  valor binario, 1 si aparece menos una vez, 0 en caso contrario).

A tener en cuenta:
- El corpus se representa mediante una matriz con una fila por texto y una columna por elemento del vocabulario.
- Es una representación del alta dimensionalidad (cada elemento del vocabulario es una característica) pero dispersa.
- El orden de las palabras en el texto se pierde.

Bag of words es uno de los algoritmos más sencillos a la hora de realizar extración de características, también, tiene muchos incovenientes como por ejemplo, la alta dimensionalidad, por todo esto, creemos que no será la opción que elijamos definitivamente. Sin embargo, la probaremos para ver si da buenos resultados

In [10]:
# Bag of words
bag_vectorizer = CountVectorizer()
counts = bag_vectorizer.fit_transform(df["comment"])
counts


<3463x7291 sparse matrix of type '<class 'numpy.int64'>'
	with 56112 stored elements in Compressed Sparse Row format>

In [11]:
# Change the ngram_range and use chars instead of words
ngram_vectorizer = CountVectorizer(analyzer="char_wb",
                                   ngram_range=(3,6))
counts = ngram_vectorizer.fit_transform(df["comment"])
counts


<3463x52027 sparse matrix of type '<class 'numpy.int64'>'
	with 730194 stored elements in Compressed Sparse Row format>

Como podemos observar en los ejemplos de bag of words obtenemos matrices de alta dimensionalidad

### Codificación TF-IDF
Consiste en una medida numérica que expresa cuán relevante es una palabra para un texto en un corpus.

La idea con TF-IDF es reducir el peso de los términos proporcionalmente al número de textos en los que aparecen. De esta forma, el valor de un término aumenta proporcionalmente al número de veces que aparece en el texto, y es compensado por su frecuencia en el corpus.

Este método podría servirnos para nuestro caso, ya que de esta forma podemos asociar ciertas palabras que se repiten en varios documentos son asociados a niveles de toxicidad. Además, esta probabilidad que nos da el algoritmo nos sirve para saber también, como de imporante es una determinada palabra o n-grama en el texto.

In [12]:
# Create a TF-IDF with ngrams from range 1 to 2
tfidf = TfidfVectorizer(ngram_range=(1,2))
# Fit with the comments 
features = tfidf.fit_transform(df["comment"])
# Get the feature extraction matrix
df_features = pd.DataFrame(features.todense(),
             columns= tfidf.get_feature_names())
# Print the first comment
print(df_original["comment"].iloc[0])
# Print the sorted by probability first row of the matrix
df_features.sort_values(by=0, axis=1, ascending=False).head(1)

Pensó: Zumo para restar.


,pens zum,zum rest,zum,rest,pens,ab,pas acept,partid disfraz,partidari,partidari descentiv,...,espure nivel,esqu,esqu problem,esquil,esquil si,esquilm,esquilm tan,esquin,esquin amanci,ñol hil
0,0.554283,0.554283,0.397122,0.364845,0.30777,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Como podemos observar en el primer ejemplo de la matriz de extracción de características.

Para el texto: 
- Pensó: Zumo para restar.

Obtenemos probabilidades acordes al texto y que reflejan que palabras son más imporantes en este texto 

## Conclusión
Una vez hemos acabado con el preprocesamiento y la extracción de caraterísticas podemos comenzar a trabajar con los modelos de machine learning.

En la siguiente práctica utilizaremos las matrices que obtenemos de los algoritmos de extracción de caraterísticas y nos quedaremos con la matriz que mejor precisión nos de a la hora de detectar la  toxicidad de un comentario. 